<a href="https://colab.research.google.com/github/abhishekshakya/BPSO-Decision-Trees-for-Software-fault-prediciton/blob/master/internPSO.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive/')
#https://ieeexplore.ieee.org/abstract/document/8011255
#from SVM https://www.youtube.com/watch?v=_PwhiWxHK8o

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/drive/


In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import random
from tqdm.notebook import tqdm
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import roc_auc_score

In [3]:
df_pc1 = pd.read_csv('/content/drive/My Drive/dataset/pc1.csv')
df_kc1 = pd.read_csv('/content/drive/My Drive/dataset/kc1.csv')
df_kc2 = pd.read_csv('/content/drive/My Drive/dataset/kc2.csv')
df_jm1 = pd.read_csv('/content/drive/My Drive/dataset/jm1.csv')

In [4]:
def normalize(x):
  x = x.astype(float)
  min = np.min(x)
  max = np.max(x)
  return (x - min)/(max-min)

x_real = df_pc1.iloc[:,:-1].apply(func= normalize, axis=0).values
y_real = df_pc1.iloc[:,-1].values.astype(int)

##Data cleaning

#####For kc2, here y feature is in object type

In [5]:
#for kc2 here change output feature from object -> bool
p = df_kc2['defects']
map = {'yes':True, 'no':False}
df_kc2['defects'] = p.map(map)


####For jm1, we have '?' randomly distributed so we need to remove them also

In [6]:
d1,d2,d3,d4,d5 = df_jm1.iloc[:,16], df_jm1.iloc[:,17], df_jm1.iloc[:,18], df_jm1.iloc[:,19], df_jm1.iloc[:,20]

for i in range(len(d1)):
  if (str)(d1[i]) == '?' or (str)(d2[i]) == '?' or (str)(d3[i]) == '?' or (str)(d4[i]) == '?' or (str)(d5[i]) == '?':
    df_jm1.drop(i,inplace=True)
    i -= 1


In [7]:
print(x_real)
print(y_real)

[[1.82724252e-03 2.96296296e-03 3.27868852e-03 ... 1.21951220e-04
  1.04895105e-03 1.70212766e-03]
 [1.66112957e-03 0.00000000e+00 0.00000000e+00 ... 0.00000000e+00
  8.74125874e-04 0.00000000e+00]
 [1.51162791e-01 5.92592593e-02 1.63934426e-02 ... 1.16463415e-01
  1.10139860e-01 6.80851064e-02]
 ...
 [8.30564784e-03 1.48148148e-02 1.63934426e-02 ... 6.09756098e-03
  5.24475524e-03 1.70212766e-02]
 [2.99003322e-02 5.18518519e-02 3.27868852e-02 ... 3.65853659e-02
  4.37062937e-02 5.95744681e-02]
 [4.31893688e-02 1.25925926e-01 9.83606557e-02 ... 7.19512195e-02
  9.52797203e-02 1.44680851e-01]]
[0 1 1 ... 0 0 0]


In [20]:
((ans==y_real).sum()*1.0)/len(ans)

0.9305680793507665

device(type='cpu')

In [115]:
# ans = ans.reshape(ans.shape[0])
corr = 0
wrng = 0
for i in range(len(ans)):
  if ans[i] == y_real[i]:
    corr += 1
  else:
    wrng += 1

acc =  (corr*1.0)/(corr+wrng)
print(corr, wrng)

1033 76


In [15]:
class PSO:
  def __init__(self,particles, dimensions, lb, ub, itrations, c1, c2, w, xr, yr):
    self.xr = xr
    self.yr = yr
    self.lb = lb
    self.ub = ub
    self.p = particles
    self.d = dimensions
    self.n = itrations
    self.c1 = c1
    self.c2 = c2
    self.w = w
    self.fitness_value = np.zeros(particles)+np.inf#p
    self.X = np.random.randint(0,2,(particles,dimensions))#pxd
    self.V = np.random.randint(0,2,(particles,dimensions))#pxd
    self.pbest = self.X#pxd
    self.gbest = np.zeros(dimensions)#d
    #---initialiastion-------------------------
    min = np.inf
    for i in range(self.p):
      fit = self.fitness(self.X[i])
      if min > fit:
        min = fit
        self.gbest = self.X[i]
    #-------------------------------------------
    self.run()

  def sigmoid(self,x):
    return 1/(1+np.exp(-x))
  
  def count(self,x):
    c = np.sum(x)
    return c

  def fitness(self,x):
    x_train = []
    y_train = self.yr

    for i in range(self.d):
      if x[i] == 1:
        x_train.append(self.xr[:,i])

    x_train = np.array(x_train).reshape((self.xr.shape[0],-1))

    if x_train.shape[1] == 0:
      return 0

    classifier = DecisionTreeClassifier()
    classifier.fit(x_train,y_train)
    chk = classifier.predict(x_train)

    acc = ((chk==y_train).sum()*1.0)/len(chk)
    return acc

  def run(self):
    for itr in tqdm(range(self.n)):
      for i in range(self.p):
        if self.fitness(self.X[i]) > self.fitness(self.pbest[i]):
          self.pbest[i] = self.X[i]

        if self.fitness(self.X[i] > self.fitness(self.gbest)):
          self.gbest = self.X[i]

        if (self.fitness(self.X[i]) == self.fitness(self.gbest) ) and (self.count(self.X[i]) < self.count(self.gbest)):
          self.gbest = self.X[i]

      r1 = random.uniform(0,1)
      r2 = random.uniform(0,1)
      for p in range(self.p):
        for q in range(self.d):
          self.V[p][q] = self.w * self.V[p][q] + self.c1*r1*(self.pbest[p][q] - self.X[p][q]) + self.c2*r2*(self.gbest[q] - self.X[p][q])

          if self.V[p][q] > self.ub:
            self.V[p][q] = self.ub
          if self.V[p][q] < self.lb:
            self.V[p][q] = self.ub

          if self.sigmoid(self.V[p][q]) > random.uniform(0,1):
            self.X[p][q] = 1
          else:
            self.X[p][q] = 0


#Testing

In [16]:
p = 50 #number of particles
d = x_real.shape[1] #total number of features
lb = 0 #min value after normalization
ub = 1 #max value after normalization
it = 30 #number of itrations
c1 = 1.5 # coefficient 1
c2 = 1.7 # coefficient 3
w = 0.7 #intertia
xr = x_real #dataset under which PSO needs to be performed
yr = y_real #dataset under which PSO needs to be performed
pp = PSO(p,d,lb,ub,it,c1,c2,w, xr, yr)

In [17]:
sum(pp.gbest)

8

###total number of features selected is 8

In [18]:
#Testing
classifier1 = DecisionTreeClassifier()
classifier2 = DecisionTreeClassifier()

In [19]:
inp1 = []
y_train = y_real

for i in range(d):
  if pp.gbest[i] == 1:
    inp1.append(x_real[:,i])

inp1 = np.array(inp1).reshape((x_real.shape[0],-1))

In [20]:
classifier1.fit(inp1, y_real)
classifier2.fit(x_real, y_real)

DecisionTreeClassifier(ccp_alpha=0.0, class_weight=None, criterion='gini',
                       max_depth=None, max_features=None, max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, presort='deprecated',
                       random_state=None, splitter='best')

In [21]:
pred1 = classifier1.predict(inp1)
pred2 = classifier2.predict(x_real)

In [22]:
from sklearn.metrics import confusion_matrix
c1 = confusion_matrix(y_real, pred1)
c2 = confusion_matrix(y_real, pred2)

In [23]:
print(c1)
print(f"accuracy->{(100*(c1[0][0]+c1[1][1])*1.0)/(c1[0][0]+c1[0][1]+c1[1][0]+c1[1][1])}")

[[1032    0]
 [   1   76]]
accuracy->99.90982867448152


In [24]:
print(c2)
print(f"accuracy->{(100*(c2[0][0]+c2[1][1])*1.0)/(c2[0][0]+c2[0][1]+c2[1][0]+c2[1][1])}")

[[1032    0]
 [   6   71]]
accuracy->99.45897204688909


In [25]:
def get(class1, class2, gbest, df):
  x = df.iloc[:,:-1].apply(func= normalize, axis=0).values
  y = df.iloc[:,-1].values.astype(int)
  ip = []
  op = y
  for i in range(d):
    if gbest[i] == 1:
      ip.append(x[:,i])

  ip = np.array(ip).reshape((x.shape[0],-1))

  prediction1 = class1.predict(ip)#feature selection
  prediction2 = class2.predict(x)#no feature selection

  c1 = confusion_matrix(op, prediction1)
  c2 = confusion_matrix(op, prediction2)

  print(f"Feature selection accuracy->{(100*(c1[0][0]+c1[1][1])*1.0)/(c1[0][0]+c1[0][1]+c1[1][0]+c1[1][1])}")
  print(f"No Feature selection accuracy->{(100*(c2[0][0]+c2[1][1])*1.0)/(c2[0][0]+c2[0][1]+c2[1][0]+c2[1][1])}")


In [26]:
get(classifier1,classifier2, pp.gbest, df_kc1)

Feature selection accuracy->81.69748696064485
No Feature selection accuracy->54.480796586059746


In [27]:
get(classifier1, classifier2, pp.gbest, df_kc2)

Feature selection accuracy->73.37164750957854
No Feature selection accuracy->72.22222222222223


In [28]:
get(classifier1, classifier2, pp.gbest, df_jm1)

Feature selection accuracy->79.63235294117646
No Feature selection accuracy->79.35661764705883
